### 相关库
- requests: 用来获取页面内容
- BeautifulSoup: 用来解析Html

#### 安装库

#### requests的7个主要方法

|方法 |说明 |
|:------|:------|
| requests.request() | 构造一个请求，支撑一下各方法的基础方法 |
| requests.get() | 获取HTML网页的主要方法，对应HTTP的GET |
| requests.head() | 获取HTML网页头的信息方法，对应HTTP的HEAD |
| requests.post() | 向HTML网页提交POST请求方法，对应HTTP的POST |
| requests.put() | 向HTML网页提交PUT请求的方法，对应HTTP的RUT |
| requests.patch() | 向HTML网页提交局部修改请求，对应于HTTP的PATCH |
| requests.delete() | 向HTML页面提交删除请求,对应HTTP的DELETE |

#### response对象的属性
|属性|说明|
|:------ |:------ |
|r.status_code|HTTP请求的返回状态|
|r.text|HTTP响应内容的字符串形式，即：url对应的页面内容|
|r.encoding|从HTTP header中猜测的响应内容编码方式|
|r.apparent_encoding|从内容中分析出的响应内容编码方式（备选编码方式）|
|r.content|HTTP响应内容的二进制形式|

#### 导入库

In [34]:
import requests
from bs4 import BeautifulSoup

## 例子：爬取bkjw.xmu.edu.cn课表
### 登录页面
#### 建立Session

In [35]:
headers={
    'User-Agent':'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/7.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0)'
}
url1="http://bkjw.xmu.edu.cn/epstar/login/index.jsp"
mySeesion=requests.session()

#### 通过post登录系统

In [36]:
userInfo={
    'userName':'metc',
    'userPassword':'1'
}
logOnSuccess=False
myRequest=mySeesion.post(url1,data=userInfo)
if myRequest.status_code==200:
    resStr=myRequest.text
    if resStr.find('/epstar/web/swms/mainframe/home/index.jsp')>0:
        logOnSuccess=True
logOnSuccess

True

### 系统页面
#### get页面信息

In [37]:
#url3="http://bkjw.xmu.edu.cn/epstar/reportJsp/show.jsp?wid=PKGL_JSZYZYXX&preview=yes&param='cond1= 1=1 and pk.xnxqdm='20191' and F_PKGL_CHECKRAWSTRINGCONFLIT(pk.zcbh,F_PKGL_GENRAWSTRINGOFJC('2','2'))=1 AND pk.xq='2' AND rm.jxl='83';cond2=1=1 AND rm.jxl='83';xnxqdm=''20191'';zcdm=''2'';week=''2'';jxldm=''83'''"
dicBuild={
    '四号楼':'83',
    '一号楼':'94'
}
dicTerm={
    '2018-2019学年 第三学期':'20183',
    '2019-2020学年 第一学期':'20191'
}
def getTableUrl(term,week,day,build,dic_build=dicBuild,dic_term=dicTerm):
    url="http://bkjw.xmu.edu.cn/epstar/reportJsp/show.jsp?wid=PKGL_JSZYZYXX&preview=yes&param='cond1= 1=1 and pk.xnxqdm='"
    url+=dic_term[term]
    url+="' and F_PKGL_CHECKRAWSTRINGCONFLIT(pk.zcbh,F_PKGL_GENRAWSTRINGOFJC('"
    url+=week
    url+="','"
    url+=week
    url+="'))=1 AND pk.xq='"
    url+=day
    url+="' AND rm.jxl='"
    url+=dic_build[build]
    url+="';cond2=1=1 AND rm.jxl='"
    url+=dic_build[build]
    url+="';xnxqdm=''"
    url+=dic_term[term]
    url+="'';zcdm=''"
    url+=week
    url+="'';week=''"
    url+=day
    url+="'';jxldm=''"
    url+=dic_build[build]
    url+="'''"
    return url

tabUrl=getTableUrl(term='2019-2020学年 第一学期',week='3',day='4',build='四号楼')


getFirstTab=False
if logOnSuccess:
    #get方法获取页面信息
    myRequest=mySeesion.get(tabUrl)
    if myRequest.status_code==200:
        #用BeautifulSoup整理页面信息
        tmpStr=myRequest.text
        if myRequest.text.find('学期教室资源占用信息')>0:
            mySoup=BeautifulSoup(myRequest.text,'lxml')
            getFirstTab=True
mySoup

<html>
<body bottommargin="0" leftmargin="0" rightmargin="0" topmargin="0">
<table border="0" cellpadding="0" cellspacing="0" id="titleTable" width="100%"><tr>
<td background="" height="22" style="font-size:13px" valign="middle" width="100%">
<table width="100%">
<tr>
<td align="center" style="font-size:12px" valign="middle" width="47%">共<span id="t_page_span"></span>页/第<span id="c_page_span"></span>页  					
						<a href="#" onclick="report1_print();return false;"><img border="no" src="/epstar/images/preview3.gif" style="vertical-align:middle"/></a>
<a href="#" onclick="report1_saveAsExcel();return false;"><img border="no" src="/epstar/images/excel.gif" style="vertical-align:middle"/></a>
<a href="#" onclick="report1_saveAsPdf();return false;"><img border="no" src="/epstar/images/pdf.gif" style="vertical-align:middle"/></a>
<a href="#" onclick="zjdy()"><img border="no" src="/epstar/images/print.gif" style="vertical-align:middle"/></a>
<a href="#" onclick="report1_saveAsWord();return f

#### 解析html
##### 解析当页课表

In [38]:
def findTable(soupTxt,tightTable=[]):
    myTRs=soupTxt.find_all('tr',height="57")
    for tr in myTRs:
        tds=tr.find_all('td')
        if len(tds)==7:
            currRow=[td.get_text() for td in tds]
            tightTable.append(currRow[1:])
    return tightTable
currTab=[]
currTab=findTable(mySoup,currTab)
currTab
#tds=myTRs[2].find_all('td') 
#currRow=[td.get_text() for td in tds]
#currRow[1:]

[['4号楼A105', '', '', '', '', ''],
 ['4号楼A301', '液压传动冯勇建院系选修课1-15(64)', '', '', '', ''],
 ['4号楼A303',
  '推拿学孟宪军院系必修课2017中医系1-15(63)',
  '药理学金鑫院系必修课2017中医系,2018护理系1-15(105)',
  '',
  '',
  ''],
 ['4号楼A305', '', '', '', '', ''],
 ['4号楼A401', '', '', '生命基础4邓小玲院系必修课2018临床医学系1-15(131)', '', ''],
 ['4号楼A402',
  '',
  '国际学院教学活动(借)院系',
  '环境与生态学院教学活动(借)院系',
  '面向对象技术与java程序设计缪克华院系选修课3-15(67)',
  ''],
 ['4号楼A404',
  '',
  '软件技术基础实验周笋院系必修课2018自动化系1-15(81)',
  'C程序设计基础曾奇斌全校性必修课1-15(单)(74)',
  '',
  ''],
 ['4号楼A405',
  '',
  '软件技术基础实验周笋院系必修课2018自动化系1-15(81)',
  'C程序设计基础曾奇斌全校性必修课1-15(单)(74)',
  '',
  '']]

##### 获取读取课表后几页的post信息

In [39]:
myForms=mySoup.find_all('form')
postForm=myForms[0]
postUrl=postForm.get('action')
postInputs=postForm.find_all('input')
postArgName=[input_.get('name') for input_ in postInputs]
postArgVal=[input_.get('value') for input_ in postInputs]
postData=dict(list(zip(postArgName,postArgVal)))
postData

{'cond1': " 1=1 and pk.xnxqdm='20191' and F_PKGL_CHECKRAWSTRINGCONFLIT(pk.zcbh,F_PKGL_GENRAWSTRINGOFJC('3','3'))=1 AND pk.xq='4' AND rm.jxl='83';cond2=1=1 AND rm.jxl='83';xnxqdm=''20191'';zcdm=''3'';week=''4'';jxldm=''83''",
 'wid': 'PKGL_JSZYZYXX',
 'reportParamsId': '100669',
 'srcId': 'A1892',
 'report1_currPage': '1',
 'report1_sessionId': 'A1894',
 'report1_cachedId': 'A1893'}

##### 获取当前页和总页数

In [43]:
import re
myScripts=mySoup.find_all('script')
for Script in myScripts:
    tmpStr=str(Script)
    if tmpStr.find('function report1_getCurrPage()')>0:
        pageInfo=re.findall('\d',tmpStr)
        if len(pageInfo)==4:
            currPage=pageInfo[1]
            totalPage=pageInfo[3]
            print(currPage+'/'+totalPage)
        


4/4


#### 使用Post方法获取后几页课表

In [41]:
for i in range(int(currPage)+1,int(totalPage)+1):
    postData['report1_currPage']=i
    myRequest=mySeesion.post(postUrl,postData)
    mySoup=BeautifulSoup(myRequest.text,'lxml')
    currTab=findTable(mySoup,currTab)
currTab

[['4号楼A105', '', '', '', '', ''],
 ['4号楼A301', '液压传动冯勇建院系选修课1-15(64)', '', '', '', ''],
 ['4号楼A303',
  '推拿学孟宪军院系必修课2017中医系1-15(63)',
  '药理学金鑫院系必修课2017中医系,2018护理系1-15(105)',
  '',
  '',
  ''],
 ['4号楼A305', '', '', '', '', ''],
 ['4号楼A401', '', '', '生命基础4邓小玲院系必修课2018临床医学系1-15(131)', '', ''],
 ['4号楼A402',
  '',
  '国际学院教学活动(借)院系',
  '环境与生态学院教学活动(借)院系',
  '面向对象技术与java程序设计缪克华院系选修课3-15(67)',
  ''],
 ['4号楼A404',
  '',
  '软件技术基础实验周笋院系必修课2018自动化系1-15(81)',
  'C程序设计基础曾奇斌全校性必修课1-15(单)(74)',
  '',
  ''],
 ['4号楼A405',
  '',
  '软件技术基础实验周笋院系必修课2018自动化系1-15(81)',
  'C程序设计基础曾奇斌全校性必修课1-15(单)(74)',
  '',
  ''],
 ['4号楼B101', '', '', '', '', ''],
 ['4号楼B304', '', '化学海洋学陈敏院系必修课20171-15(单)(36)', '', '', ''],
 ['4号楼B305', '管理信息系统B曾一锋院系必修课2016自动化系1-8(10)', '', '', '', ''],
 ['4号楼B306', '', '', '', '', ''],
 ['4号楼B307', '', '', '', '', ''],
 ['4号楼B308', '', '', '', '', ''],
 ['4号楼B309', '', '', '', '', ''],
 ['4号楼B310', '', '', '', '', ''],
 ['4号楼B311', '', '', '', '', ''],
 ['4号楼B312', '', '', '', '', ''],
 ['4

#### 退出登录

In [42]:
logoutUrl='http://bkjw.xmu.edu.cn/epstar/login/logout.jsp'
mySeesion.get(logoutUrl)
mySeesion.close()